# Project: Binary Bernoulli Bandit Risk Aware Ticker Picker
In the binary Bernoulli bandit ticker picker example, we evaluated the return of the possible tickers over investment periods of length `buffersize::Int64.` However, we failed to consider the risk associated with these tickers. In this project, students will refactor the example code to include the risk for each ticker.

### Learning objectives and tasks
Fill me in.

## Setup
We set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our lab problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl).

In [3]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CH

## Prerequisites: Load ticker-picker save file
Let's begin by loading the [HDF5 encoded saved file](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) generated in the worked example using [a `load(...)`  method exported by the JLD2.jl package](https://github.com/JuliaIO/JLD2.jl). First, we specify the path to the saved file in the `path_to_save_file::String` variable:

In [5]:
number_of_agents = 5000; # how many agents do we want
path_to_save_file = joinpath(_PATH_TO_DATA, "TickerPickerAgent-Simulation-State-N-$(number_of_agents).jld2");

then we [call the `load(...)` method exported by the JLD2.jl package](https://github.com/JuliaIO/JLD2.jl), which reads the binary saved file and returns the saved data as a dictionary; we assign the data to the `saved_state_dict::Dict{String, Any}` variable:

In [7]:
saved_state_dict = load(path_to_save_file)

Dict{String, Any} with 7 entries:
  "risk_free_rate" => 0.047
  "buffersize"     => 5
  "data"           => Dict{String, DataFrame}("TPR"=>1508×8 DataFrame…
  "preferences"    => Beta[Beta{Float64}(α=7.0, β=10.0) Beta{Float64}(α=3.0, β=…
  "tickers"        => ["MRK", "JNJ", "MET", "NFLX", "AAPL", "AMD", "MU", "INTC"…
  "banditmodel"    => MyEpsilonSamplingBanditModel([1.0, 1.0, 1.0, 1.0, 1.0, 1.…
  "Δt"             => 0.00396825

## Task 1: Compute the risk measure dictionary for my ticker list
Fill me in

In [21]:
bandit_model = saved_state_dict["banditmodel"];

## Task 2:  Build risk-aware ticker-picker agent and world model
Fill me in

### TODO: Setup a MyTickerPickerWorldModel instance
Now that we have set up the bandit model, we construct [a `MyTickerPickerWorldModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.MyTickerPickerWorldModel) which holds data about the world which we sample. The `world(...)` function takes problem data and returns a binary reward. First, we formulate the `world(...)` function, which takes the `start::Int64` index, an action $a\in\mathcal{A}$ value, and the `worldmodel::MyTickerPickerWorldModel.` The `world(...)` function returns a binary reward $r\in\left\{0,1\right\}$.

#### Summary of world function
`Unhide` the code block to see a reference implementation of the `world(...)` function. 
* We compare the wealth generated by investing `1 USD` in a fund that returns the risk-free rate for `buffersize::Int64` periods of $\Delta{t}$ duration (denoted as `W_risk_free`) versus `1 USD` in a `ticker` over the same period (denoted as `W_ticker`). If `W_ticker > W_risk_free`, the `world(...)` function returns `1`, i.e., success; otherwise, the `world(...)` function returns `0.`

## Task 3: Run a collection of ticker-picker agents and examine their preferences
Fill me in

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.